In [5]:
from warcio.archiveiterator import ArchiveIterator
import re
import requests
import sys
from bs4 import BeautifulSoup

In [3]:
# !pip install warcio
!pip install beautifulsoup4


In [12]:
# regex = re.compile(
#     "(covid\.be/|corona\.com/(Virus\?(.*\&)?v=|(economy|v)/))([^?&\"'>]+)"
# )

#https://arxiv.org/pdf/2102.09507.pdf
#https://towardsdatascience.com/web-scraping-news-articles-in-python-9dd605799558 
all_url = set()
def get_url(seg_url):
    regex = re.compile(
        "((?=.*pandemic*)|(?=.*covid*)|(?=.*lockdown*)) "
    )

    entries = 0
    matching_entries = 0
    hits = 0

    #file_name = "http://commoncrawl.s3.amazonaws.com/crawl-data/CC-MAIN-2019-30/segments/1563195523840.34/warc/CC-MAIN-20190715175205-20190715200159-00000.warc.gz"

    file_name = "https://commoncrawl.s3.amazonaws.com/" + seg_url

    # if len(sys.argv) > 1:
    #     file_name = sys.argv[1]

    stream = None

    if file_name.startswith("http://") or file_name.startswith(
        "https://"
    ):
        stream = requests.get(file_name, stream=True).raw
    else:
        stream = open(file_name, 'rb')
    count = 0


    for record in ArchiveIterator(stream):
        content_str = ""
        if record.rec_type == "warcinfo":
            continue
        #print(record.content_stream())
        #break
        if not ".com/" in record.rec_headers.get_header(
            "WARC-Target-URI"
        ):
            continue

        entries = entries + 1
        contents = (
            record.content_stream()
            .read()
            .decode("utf-8", "replace")
        )
        url  = record.rec_headers.get_header('WARC-Target-URI')

        # soup = BeautifulSoup(contents, 'html5lib')
        # htmlcontent = soup.find_all()
        # for elem in htmlcontent:
        #   content_str += elem.get_text()
        #   count +=1
    
        m = regex.search(contents,  re.IGNORECASE)
        if m:
            matching_entries = matching_entries + 1
            hits = hits + 1
            # m = regex.search(content_str, m.end())


        while m:
          # m = regex.search(content_str, m.end())
            hits = hits + 1
            all_url.append(url)
            print(url)
      
        if len(all_url) == 1000:
          break

    print(
        "Python: "
        + str(hits)
        + " matches in "
        + str(matching_entries)
        + "/"
        + str(entries)
    )


In [ ]:

Year_2020_url = ['crawl-data/CC-MAIN-2020-45/segments/1603107866404.1/warc/CC-MAIN-20201019203523-20201019233523-00177.warc.gz',
'crawl-data/CC-MAIN-2020-45/segments/1603107866404.1/warc/CC-MAIN-20201019203523-20201019233523-00178.warc.gz',
'crawl-data/CC-MAIN-2020-45/segments/1603107866404.1/warc/CC-MAIN-20201019203523-20201019233523-00179.warc.gz',
'crawl-data/CC-MAIN-2020-45/segments/1603107866404.1/warc/CC-MAIN-20201019203523-20201019233523-00180.warc.gz',
'crawl-data/CC-MAIN-2020-45/segments/1603107866404.1/warc/CC-MAIN-20201019203523-20201019233523-00181.warc.gz',
'crawl-data/CC-MAIN-2020-45/segments/1603107866404.1/warc/CC-MAIN-20201019203523-20201019233523-00182.warc.gz',
'crawl-data/CC-MAIN-2020-45/segments/1603107866404.1/warc/CC-MAIN-20201019203523-20201019233523-00183.warc.gz',
'crawl-data/CC-MAIN-2020-45/segments/1603107866404.1/warc/CC-MAIN-20201019203523-20201019233523-00184.warc.gz',
'crawl-data/CC-MAIN-2020-45/segments/1603107866404.1/warc/CC-MAIN-20201019203523-20201019233523-00185.warc.gz',
'crawl-data/CC-MAIN-2020-45/segments/1603107866404.1/warc/CC-MAIN-20201019203523-20201019233523-00186.warc.gz']

for url in Year_2020_url:
  get_url(url)



In [ ]:
print(len(all_url))

In [ ]:
from spacy.matcher import PhraseMatcher
from scipy import spatial

# method for searching keyword from the text
def search_for_keyword(keyword, doc_obj, nlp):
    phrase_matcher = PhraseMatcher(nlp.vocab)
    phrase_list = [nlp(keyword)]
    phrase_matcher.add("Text Extractor", None, *phrase_list)

    matched_items = phrase_matcher(doc_obj)

    matched_text = []
    for match_id, start, end in matched_items:
        text = nlp.vocab.strings[match_id]
        span = doc_obj[start: end]
        matched_text.append(span.sent.text)

In [ ]:
!git clone https://github.com/EdwardJRoss/job-advert-analysis.git 

Cloning into 'job-advert-analysis'...
remote: Enumerating objects: 729, done.
remote: Counting objects: 100% (729/729), done.
remote: Compressing objects: 100% (367/367), done.
remote: Total 729 (delta 484), reused 593 (delta 349), pack-reused 0
Receiving objects: 100% (729/729), 977.47 KiB | 14.37 MiB/s, done.
Resolving deltas: 100% (484/484), done.


In [ ]:
pip install mypy-extensions

In [ ]:
class CommonCrawlDatasource():

    query: str
    query_filters: List[str] = []
    # Number of threads for parallel
   #nthread: int = _DEFAULT_NTHREAD
    disable_progress: bool = False

    raw_extension = ".warc.gz"

    sources = {
        "CC-MAIN-2020-50": "https://index.commoncrawl.org/CC-MAIN-2020-50-index",
        "CC-MAIN-2020-45": "https://index.commoncrawl.org/CC-MAIN-2020-45-index",
        "CC-MAIN-2020-40": "https://index.commoncrawl.org/CC-MAIN-2020-40-index",
        "CC-MAIN-2020-34": "https://index.commoncrawl.org/CC-MAIN-2020-34-index",
        "CC-MAIN-2020-29": "https://index.commoncrawl.org/CC-MAIN-2020-29-index",
        "CC-MAIN-2020-24": "https://index.commoncrawl.org/CC-MAIN-2020-24-index",
        "CC-MAIN-2020-16": "https://index.commoncrawl.org/CC-MAIN-2020-16-index",
        "CC-MAIN-2020-10": "https://index.commoncrawl.org/CC-MAIN-2020-10-index",
        "CC-MAIN-2020-05": "https://index.commoncrawl.org/CC-MAIN-2020-05-index",
    }

    def fetch_source_rows(self, source: str) -> List[CrawlResultDict]:
        return fetch_source_rows(source, self.query, self.query_filters)

    def download_one(self, path: Path, source: str) -> None:
        logging.info(f"Fetching {source} for {self.name}")
        source_rows = self.fetch_source_rows(source)
        with AtomicFileWriter(path) as output:
            writer = WARCWriter(output, gzip=True)
            logging.info(f"Downloading {source}")
            for warc in fetch_all_cc(source_rows, self.nthread, self.disable_progress):
                writer.write_record(warc)

    def extract(self, html: bytes, uri: str, view_date: str) -> List[Dict[Any, Any]]:
        pass

    def extract_one(self, path: Path) -> Generator[Dict[Any, Any], None, None]:
        for warc in read_warc_responses(path):
            html = warc.content_stream().read()
            uri = warc.rec_headers["WARC-Target-URI"]
            view_date = warc.rec_headers["WARC-Date"]

            assert uri is not None
            assert view_date is not None

            for result in self.extract(html, uri, view_date):
                yield result
